In [1]:
from jisho_api.kanji import Kanji
from jisho_api.word import Word
import pandas as pd
import time
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
from sqlalchemy import create_engine

In [2]:
r = Kanji.request('現')

In [3]:
k = r.data

print("Kanji:", k.kanji)
print("Strokes:", k.strokes)
print("Meanings:", k.main_meanings)
print("Kun readings:", k.main_readings.kun)
print("On readings:", k.main_readings.on)
# From metadata
if k.meta.education:
    print("Grade:", k.meta.education.grade)
    print("JLPT:", k.meta.education.jlpt)
    print("Newspaper rank:", k.meta.education.newspaper_rank)
# Radical info
print("Radical meaning:", k.radical.meaning)
print("Radical basis:", k.radical.basis)
# Reading examples
if k.reading_examples:
    if k.reading_examples.kun:
        print("\nKun Reading Examples:")
        for ex in k.reading_examples.kun:
            print(f"{ex.kanji} ({ex.reading}) → {ex.meanings}")
    if k.reading_examples.on:
        print("\nOn Reading Examples:")
        for ex in k.reading_examples.on:
            print(f"{ex.kanji} ({ex.reading}) → {ex.meanings}")


Kanji: 現
Strokes: 11
Meanings: ['present', 'existing', 'actual']
Kun readings: ['あらわ.れる', 'あらわ.す', 'うつつ', 'うつ.つ']
On readings: ['ゲン']
Grade: None
JLPT: None
Newspaper rank: None
Radical meaning: jade
Radical basis: jade

Kun Reading Examples:
現れる (あらわれる) → ['to appear', 'to come in sight', 'to become visible', 'to come out', 'to embody', 'to materialize', 'to materialise', 'to be expressed (e.g. emotions)', 'to become apparent (e.g. trends', 'effects)']
表す (あらわす) → ['to represent', 'to signify', 'to stand for', 'to reveal', 'to show', 'to display', 'to express', 'to make widely known']
現つ (うつつ) → ['reality', 'consciousness']
現責 (うつつぜめ) → ['sleep deprivation (Edo-period torture method)']
現つ (うつつ) → ['reality', 'consciousness']
現責 (うつつぜめ) → ['sleep deprivation (Edo-period torture method)']

On Reading Examples:
現 (ゲン) → ['present (e.g. government', 'administration)', 'current', 'existing']
現役 (ゲンエキ) → ['active duty', 'active service', 'student taking (university) entrance exams while sti

In [4]:
def is_kanji(char):
    return (
        '\u4e00' <= char <= '\u9faf' or  # CJK Unified Ideographs
        '\u3400' <= char <= '\u4dbf'    # CJK Unified Ideographs Extension A
    )

In [5]:
jlpt_df=pd.read_csv('dataset/jlpt_vocab.csv').rename(columns={
    'Original':'word',
    'Furigana':'furigana',
    'English':'translation',
    'JLPT Level':'jlpt_level'
})
jlpt_df.head()

,word,furigana,translation,jlpt_level
0,現像,げんぞう,developing (film),N1
1,原則,げんそく,"principle, general rule",N1
2,見地,けんち,point of view,N1
3,現地,げんち,"actual place, local",N1
4,限定,げんてい,"limit, restriction",N1


In [6]:
kanji_list = jlpt_df["word"].apply(
    lambda w: [c for c in w if is_kanji(c)]
)

In [7]:
all_kanji = [kanji for sublist in kanji_list for kanji in sublist]

In [8]:
kanji_counts = Counter(all_kanji)

In [9]:
kanji_df = pd.DataFrame.from_dict(kanji_counts, orient="index", columns=["count"]).reset_index().rename(columns={'index':'kanji'})

In [30]:
kana_df=pd.read_csv('dataset/kana_romaji.csv')

In [11]:
kana_map = {}

for _, row in kana_df.iterrows():
    kana_map[row["kana"]] = row["romaji"]


In [12]:
digraphs = {k: v for k, v in kana_map.items() if len(k) == 2}
monographs = {k: v for k, v in kana_map.items() if len(k) == 1}


In [13]:
def kana_to_romaji(text):
    if not isinstance(text, str):
        return None
    
    i = 0
    result = []
    length = len(text)

    while i < length:
        # Handle small tsu: っ or ッ
        if text[i] in ["っ", "ッ"]:
            # Look ahead to next kana (digraph or monograph)
            if i + 2 <= length and text[i+1:i+3] in digraphs:
                # double first consonant of romaji
                next_romaji = digraphs[text[i+1:i+3]]
                result.append(next_romaji[0])
                i += 1
                continue
            elif i + 1 < length and text[i+1] in monographs:
                next_romaji = monographs[text[i+1]]
                result.append(next_romaji[0])
                i += 1
                continue
            else:
                # Lone small tsu — rare — ignore
                i += 1
                continue

        # Digraph (e.g. ちゃ, きゃ)
        if i + 2 <= length and text[i:i+2] in digraphs:
            result.append(digraphs[text[i:i+2]])
            i += 2
            continue

        # Monograph
        if text[i] in monographs:
            result.append(monographs[text[i]])
            i += 1
            continue

        # Unknown character (kanji or punctuation) → keep as-is or skip
        result.append(text[i])
        i += 1

    return "".join(result)


In [14]:
jlpt_df["romaji"] = jlpt_df["furigana"].apply(kana_to_romaji)

In [15]:
jlpt_df["num_characters"] = jlpt_df["word"].apply(lambda x: len(str(x)))


In [17]:
jlpt_df.head()

,word,furigana,translation,jlpt_level,romaji,num_characters
0,現像,げんぞう,developing (film),N1,genzou,2
1,原則,げんそく,"principle, general rule",N1,gensoku,2
2,見地,けんち,point of view,N1,kenchi,2
3,現地,げんち,"actual place, local",N1,genchi,2
4,限定,げんてい,"limit, restriction",N1,gentei,2


In [18]:
kanji_df=pd.read_csv('dataset/kanji_2.csv')

In [19]:
kanji_df.head()

,kanji,count,strokes,translation,kun_readings,on_readings,radical_basis,radical_meaning,kun_romaji,on_romaji
0,現,19,11,"present, existing, actual","あらわ.れる, あらわ.す, うつつ, うつ.つ",ゲン,jade,jade,"arawa.reru, arawa.su, utsutsu, utsu.tsu",gen
1,像,5,14,"statue, picture, image, figure, portrait",NaN,ゾウ,人,man,NaN,zou
2,原,20,10,"meadow, original, primitive, field, plain, pra...",はら,ゲン,厂,cliff,hara,gen
3,則,4,9,"rule, law, follow, based on, model after","のっと.る, のり, すなわち",ソク,刀,knife,"notto.ru, nori, sunawachi",soku
4,見,50,7,"see, hopes, chances, idea, opinion, look at, v...","み.る, み.える, み.せる",ケン,見,see,"mi.ru, mi.eru, mi.seru",ken


In [20]:
kana_df.head()

,kana,romaji,type,stroke_count
0,あ,a,hiragana,3
1,い,i,hiragana,2
2,う,u,hiragana,2
3,え,e,hiragana,2
4,お,o,hiragana,3


In [32]:
# Create a map for Kanji stroke counts (Character -> Count)
kanji_map = kanji_df.set_index('kanji')['strokes'].to_dict()

# Create a map for Kana stroke counts (Character -> Count)
kana_map = kana_df.set_index('kana')['stroke_count'].to_dict()

# Combine both maps into a single lookup dictionary
stroke_count_map = {**kanji_map, **kana_map}

print(f"Total characters with stroke counts available: {len(stroke_count_map)}")

Total characters with stroke counts available: 2210


In [ ]:
def calculate_word_stroke_count(word, stroke_map):
    """
    Calculates the total stroke count for a Japanese word by summing strokes
    of every individual character element.
    """
    total_strokes = 0
    for char in str(word): # Ensure word is treated as a string
        # Retrieves stroke count from the map, defaults to 0 if the character is not found
        count = stroke_map.get(char, 0) 
        total_strokes += count
    return total_strokes

jlpt_df['stroke_count'] = jlpt_df['word'].apply(
    lambda x: calculate_word_stroke_count(x, stroke_count_map)
)

In [38]:
jlpt_df.to_csv('dataset/jlpt_vocab_clean.csv')

In [17]:
def get_kanji_info_safe(kanji):
    try:
        r = Kanji.request(kanji)
        return {
            "strokes": r.data.strokes,
            "translation": r.data.main_meanings,
            "kun_readings": r.data.main_readings.kun,
            "on_readings": r.data.main_readings.on,
            "radical_basis": r.data.radical.basis,
            "radical_meaning": r.data.radical.meaning
        }
    except:
        return {
            "strokes": None,
            "translation": None,
            "kun_readings": None,
            "on_readings": None,
            "radical_basis": None,
            "radical_meaning": None
        }

kanji_list = kanji_df["kanji"].tolist()

with ThreadPoolExecutor(max_workers=10) as ex:
    results = list(ex.map(get_kanji_info_safe, kanji_list))

kanji_info_df = pd.DataFrame(results)

kanji_df = pd.concat([kanji_df, kanji_info_df], axis=1)


In [18]:
kanji_df.head()

,kanji,count,strokes,translation,kun_readings,on_readings,radical_basis,radical_meaning
0,現,19,11,"[present, existing, actual]","[あらわ.れる, あらわ.す, うつつ, うつ.つ]",[ゲン],jade,jade
1,像,5,14,"[statue, picture, image, figure, portrait]",None,[ゾウ],人,"man,"
2,原,20,10,"[meadow, original, primitive, field, plain, pr...",[はら],[ゲン],厂,cliff
3,則,4,9,"[rule, law, follow, based on, model after]","[のっと.る, のり, すなわち]",[ソク],刀,"knife,"
4,見,50,7,"[see, hopes, chances, idea, opinion, look at, ...","[み.る, み.える, み.せる]",[ケン],見,see


In [19]:
kanji_df['translation']=kanji_df['translation'].apply(", ".join)
kanji_df['kun_readings']=kanji_df['kun_readings'].astype(str).str.replace("'", "").str.lstrip('[').str.rstrip(']')
kanji_df['on_readings']=kanji_df['on_readings'].astype(str).str.replace("'", "").str.lstrip('[').str.rstrip(']')
kanji_df['radical_meaning']=kanji_df['radical_meaning'].astype(str).str.rstrip(",")
kanji_df['kun_romaji']=kanji_df['kun_readings'].apply(kana_to_romaji)
kanji_df['on_romaji']=kanji_df['on_readings'].apply(kana_to_romaji)

In [20]:
kanji_df.head()

,kanji,count,strokes,translation,kun_readings,on_readings,radical_basis,radical_meaning,kun_romaji,on_romaji
0,現,19,11,"present, existing, actual","あらわ.れる, あらわ.す, うつつ, うつ.つ",ゲン,jade,jade,"arawa.reru, arawa.su, utsutsu, utsu.tsu",gen
1,像,5,14,"statue, picture, image, figure, portrait",None,ゾウ,人,man,None,zou
2,原,20,10,"meadow, original, primitive, field, plain, pra...",はら,ゲン,厂,cliff,hara,gen
3,則,4,9,"rule, law, follow, based on, model after","のっと.る, のり, すなわち",ソク,刀,knife,"notto.ru, nori, sunawachi",soku
4,見,50,7,"see, hopes, chances, idea, opinion, look at, v...","み.る, み.える, み.せる",ケン,見,see,"mi.ru, mi.eru, mi.seru",ken


In [21]:
kanji_df.to_csv('dataset/kanji_2.csv', index=False)

In [42]:
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError

# --- CONFIRMED NEW CREDENTIALS ---
DB_USER = 'ironhack_user'
DB_PASSWORD = 'ironhack'  # Use the password you set in Workbench
DB_HOST = '127.0.0.1'              # Use 127.0.0.1 or localhost (127.0.0.1 is safer)
DB_PORT = '3306'       
DB_NAME = 'ironhack_db'            # Use the new database name

# The complete connection string (URL) using the pymysql driver
DB_URL = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create the engine
engine = create_engine(DB_URL)

# Test the connection
try:
    with engine.connect() as connection:
        print("✅ Success! Python is connected to the new ironhack_db.")
        
except OperationalError as e:
    print("❌ Connection Failed. Double-check your user, password, and database name.")
    print(f"Error details: {e}")

✅ Success! Python is connected to the new ironhack_db.


In [39]:
jlpt_df.head()

,word,furigana,translation,jlpt_level,romaji,num_characters,stroke_count
0,現像,げんぞう,developing (film),N1,genzou,2,25
1,原則,げんそく,"principle, general rule",N1,gensoku,2,19
2,見地,けんち,point of view,N1,kenchi,2,13
3,現地,げんち,"actual place, local",N1,genchi,2,17
4,限定,げんてい,"limit, restriction",N1,gentei,2,17


In [50]:
from sqlalchemy import String, Integer

jlpt_sql_dtypes={
    'index': Integer,
    'word': String(100),
    'furigana': String(100),
    'translation': String(300),
    'jlpt_level': String(100),
    'romaji': String(100),
    'num_characters': Integer,
    'stroke_count': Integer
}

try:
    jlpt_df.to_sql(
        name='jlpt_vocabulary',     # The name of the MySQL table
        con=engine,                 # The connection engine
        if_exists='replace',        # Drop table if it exists, then create and insert
        index=True,
        dtype=jlpt_sql_dtypes
    )
    print("✅ Successfully created and loaded table 'jlpt_words' in MySQL.")

except Exception as e:
    print(f"❌ Failed to create table: {e}")

✅ Successfully created and loaded table 'jlpt_words' in MySQL.


In [51]:
kanji_df.head()

,kanji,count,strokes,translation,kun_readings,on_readings,radical_basis,radical_meaning,kun_romaji,on_romaji
0,現,19,11,"present, existing, actual","あらわ.れる, あらわ.す, うつつ, うつ.つ",ゲン,jade,jade,"arawa.reru, arawa.su, utsutsu, utsu.tsu",gen
1,像,5,14,"statue, picture, image, figure, portrait",NaN,ゾウ,人,man,NaN,zou
2,原,20,10,"meadow, original, primitive, field, plain, pra...",はら,ゲン,厂,cliff,hara,gen
3,則,4,9,"rule, law, follow, based on, model after","のっと.る, のり, すなわち",ソク,刀,knife,"notto.ru, nori, sunawachi",soku
4,見,50,7,"see, hopes, chances, idea, opinion, look at, v...","み.る, み.える, み.せる",ケン,見,see,"mi.ru, mi.eru, mi.seru",ken


In [52]:
kanji_sql_dtypes={
    'index': Integer,
    'kanji': String(1),
    'kun_readings': String(100),
    'kun_romaji': String(200),
    'on_readings': String(100),
    'on_romaji': String(200),
    'translation': String(300),
    'radical_basis': String(20),
    'radical_meaning': String(100),
    'count': Integer,
    'strokes': Integer
}

try:
    kanji_df.to_sql(
        name='kanji',     # The name of the MySQL table
        con=engine,                 # The connection engine
        if_exists='replace',        # Drop table if it exists, then create and insert
        index=True,
        dtype=kanji_sql_dtypes
    )
    print("✅ Successfully created and loaded table 'kanji' in MySQL.")

except Exception as e:
    print(f"❌ Failed to create table: {e}")

✅ Successfully created and loaded table 'kanji' in MySQL.


In [53]:
kana_df.head()

,kana,romaji,type,stroke_count
0,あ,a,hiragana,3
1,い,i,hiragana,2
2,う,u,hiragana,2
3,え,e,hiragana,2
4,お,o,hiragana,3


In [56]:
kana_sql_dtypes={
    'index': Integer,
    'kana': String(2),
    'romaji': String(3),
    'type': String(8),
    'stroke_count': Integer
}

try:
    kana_df.to_sql(
        name='kana',     # The name of the MySQL table
        con=engine,                 # The connection engine
        if_exists='replace',        # Drop table if it exists, then create and insert
        index=True,
        dtype=kana_sql_dtypes
    )
    print("✅ Successfully created and loaded table 'kana' in MySQL.")

except Exception as e:
    print(f"❌ Failed to create table: {e}")

✅ Successfully created and loaded table 'kana' in MySQL.
